In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import  LinearRegression
from sklearn.tree import  DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost.sklearn import XGBRegressor
import joblib
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from xgboost import XGBRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import uniform, randint

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression  
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor


import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("/kaggle/input/vehicle-sales-data/car_prices.csv")



* understand the data : check columns, data types , size of data

In [ ]:
print(df.columns)



In [ ]:
print(df.head())

In [ ]:
print(df.info())

In [ ]:
df.describe()

In [ ]:
df.describe(include='O')

* check null values percentage and duplication

In [ ]:
df.duplicated().sum()

In [ ]:
(df.isnull().sum() / len(df) ) * 100

* handling null values

In [ ]:
df['model'].fillna(df['model'].mode()[0] , inplace=True)
df['trim'].fillna(df['trim'].mode()[0] , inplace=True)
df['transmission'].fillna('no_transmission' ,inplace=True)
df['condition'].fillna(df['condition'].mode()[0] , inplace=True)
df['odometer'].fillna(df['odometer'].mean() , inplace=True)
df['color'].fillna(method= 'ffill' , inplace=True)
df['interior'].fillna(method= 'ffill' , inplace=True)
df['mmr'].fillna(df['mmr'].mean() , inplace=True)
df['sellingprice'].fillna(df['sellingprice'].mean() , inplace=True)
df.dropna(inplace=True)


In [ ]:
df.isnull().sum()

* convert 'saledate' to time series

In [ ]:

df['saledate'] = pd.to_datetime(df['saledate'] , utc = True , errors='coerce')

df['saledate'].dtype

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['condition'] = df['condition'] / 10
df.dropna(inplace=True)

* handling unique values ( condition where we have the same value but difference upper and lower cases like: Kia and kia)

In [ ]:
df.nunique()

In [ ]:
df['make'] = df['make'].str.upper()
df['model']=df['model'].str.upper()
df['trim']=df['trim'].str.upper()
df['body']=df['body'].str.upper()
df['state']=df['state'].str.upper()
df['color']=df['color'].str.upper()
df['interior']=df['interior'].str.upper()

In [ ]:
df.nunique()

In [ ]:
df.vin.duplicated().sum()

Data Visualization

In [ ]:
num_cols = ['year','odometer','sellingprice','mmr','condition']

corr = df[num_cols].corr()
sns.heatmap(corr, annot=True)
plt.show()

* check outliers by box plot

In [ ]:
fig , ax = plt.subplots(5, 1, figsize=(14,12))
for i, ii in enumerate(num_cols):
    ax[i].boxplot(df[ii], vert=False, patch_artist=True)
    ax[i].set_ylabel(ii)
plt.show()


In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(data=df , x='odometer' , kde=True)
plt.title('Odometer Distribution')
plt.xlabel('odometer in miles')
plt.ylabel('Range')
plt.show()

In [ ]:
q1 = df['odometer'].quantile(0.25)
q3 = df['odometer'].quantile(0.75)
IQR = q3 - q1
lower_bound = q1 - 1.5 * IQR
upper_bound = q3 + 1.5 * IQR
df['odometer']=df['odometer'].clip(lower=lower_bound,upper=upper_bound)

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(x=df['odometer'],color="green",kde=True)
plt.xlabel('odometer')
plt.ylabel('Range')
plt.title('odometer Distribution after outlier handling')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(data=df, x='sellingprice', kde=True)
plt.title('Selling Prices')
plt.xlabel('selling prices')
plt.ylabel('Range of prices')
plt.show()

In [ ]:
q1 = df['sellingprice'].quantile(0.25)
q3 = df['sellingprice'].quantile(0.75)
IQR = q3 - q1
lower_bound = q1 - 1.5 * IQR
upper_bound = q3 + 1.5 * IQR
df['sellingprice']=df['sellingprice'].clip(lower=lower_bound,upper=upper_bound)


In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(x=df['sellingprice'],color="Skyblue",kde=True)
plt.xlabel('sellingprice')
plt.ylabel('Range')
plt.title('sellingprice Distribution outlier handling')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(data=df, x='mmr', kde=True)
plt.title('MMR (Market Prices)')
plt.xlabel('Market price')
plt.ylabel('Range of prices')
plt.show()

In [ ]:
q1 = df['mmr'].quantile(0.25)
q3 = df['mmr'].quantile(0.75)
IQR = q3 - q1
lower_bound = q1 - 1.5 * IQR
upper_bound = q3 + 1.5 * IQR
df['mmr']=df['mmr'].clip(lower=lower_bound,upper=upper_bound)

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(x=df['mmr'],color="purple",kde=True)
plt.xlabel('mmr')
plt.ylabel('Range')
plt.title('mmr Distribution after outlier handling')
plt.show()

In [ ]:
plt.figure( figsize = (12,6) )
sns.histplot(data=df , x='year' )
plt.title("Year Distribution")
plt.xlabel("Year")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(data=df, x='condition',bins=20, kde=True)
plt.title("Distribution of Condition")
plt.xlabel("Condition")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(data=df , x=df['saledate'].dt.to_period('M').astype(str))
plt.xlabel("Year - Month")
plt.ylabel("Range")
plt.title("Number of Car Sales Per Years")
plt.show()

In [ ]:
sorted_brands = df.groupby("make")["sellingprice"].max().sort_values(ascending=False).index

plt.figure(figsize=(20, 10))
sns.barplot(x="make", y="sellingprice", data=df, palette="magma", order=sorted_brands)
plt.title("Selling Price by Make (Sorted by Max Price)")
plt.xlabel("Make")
plt.ylabel("Selling Price")
plt.xticks(rotation=70)
plt.show()



In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x='year', y='sellingprice', data=df, palette="magma")
plt.title('Selling price through years')
plt.xlabel('year')
plt.ylabel('sellingprice')
plt.xticks(rotation=70)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))

transmission_counts = df[df["transmission"] != "no_transmission"]["transmission"].value_counts()

plt.figure(figsize=(7, 7))
plt.pie(transmission_counts, labels=transmission_counts.index, autopct="%1.1f%%")

plt.title("Distribution of Transmission Types")
plt.show()


* some statistic questions:

which car brand has the highest sales :

In [ ]:
most_sold_make =df['make'].value_counts().idxmax()
print("The most sold brand is: ",most_sold_make)

cheapest_make = make_avg_price.idxmin()
print(f"The car make with the lowest average price is: {cheapest_make}")

what is the average selling price :

In [ ]:
avg_price = df['sellingprice'].mean()
print('avrage selling price is : ', avg_price)

Top car brands have average selling prices :

In [ ]:
top_avg_price = df.groupby('make')['sellingprice'].mean().sort_values(ascending=False)
print('top brands have avrage prices : \n',top_avg_price.head(10))

which brands that have the lowest average price:

In [ ]:
cheapest_make = top_avg_price.tail(10).sort_values()
print("the brand that has the lowest average price is:" ,cheapest_make)

 which year had the highest number of car sales :

In [ ]:
most_sales_year = df['saledate'].dt.year.value_counts().idxmax()
print("The year with the highest number of sales is:", most_sales_year)

In [ ]:
df.info()

## Modelling

In [ ]:
# todo get the first 100 row of df
# TODO: REMOVE THE FOLLOWING LINE
# df = df.head(100)

In [ ]:
# feature = ["year" , "odometer", "condition" , "mmr"]
# target= "sellingprice"
# X = df[feature]
# y = df[target]

X = df.drop(columns=['sellingprice'])
y = df['sellingprice']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [ ]:
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('scaler', StandardScaler())
        ]), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)


### Linear Regression

In [ ]:

lr = LinearRegression()

# Reuse the same preprocessing pipeline
lr_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', lr)
])

# Fit the model
lr_pipeline.fit(X_train, y_train)

# Predict
lr_predictions = lr_pipeline.predict(X_test)

# Evaluate
r2_lr = r2_score(y_test, lr_predictions)
mae_lr = mean_absolute_error(y_test, lr_predictions)
rmse_lr = np.sqrt(mean_squared_error(y_test, lr_predictions))

# Print results
print(f"Linear Regression R² Score: {r2_lr:.4f}")
print(f"Linear Regression MAE: {mae_lr:.4f}")
print(f"Linear Regression RMSE: {rmse_lr:.4f}")
joblib.dump(lr_pipeline, 'linear_regression_pipeline.joblib')


### XGBoost

In [ ]:
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', xgb)
])


param_dist = {
    'model__n_estimators': randint(50, 200),
    'model__max_depth': randint(3, 10),
    'model__learning_rate': uniform(0.01, 0.2),
    'model__subsample': uniform(0.6, 0.4),
    'model__colsample_bytree': uniform(0.6, 0.4)
}

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1,
    random_state=42
)


random_search.fit(X_train, y_train)
predictions = random_search.predict(X_test)

print("Best parameters:", random_search.best_params_)
from sklearn.metrics import r2_score


r2 = r2_score(y_test, predictions)
print(f"R² Score (Accuracy): {r2:.4f}")
joblib.dump(random_search.best_estimator_, 'best_xgb_pipeline.joblib')


### RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(random_state=42)

# Build the pipeline
rf_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', rf)
])

# Define hyperparameter distribution
param_dist_rf = {
    'model__n_estimators': randint(100, 300),
    'model__max_depth': randint(3, 15),
    'model__min_samples_split': randint(2, 10),
    'model__min_samples_leaf': randint(1, 10),
    'model__max_features': ['auto', 'sqrt', 'log2']
}

# Random search
rf_random_search = RandomizedSearchCV(
    rf_pipeline,
    param_distributions=param_dist_rf,
    n_iter=20,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Fit
rf_random_search.fit(X_train, y_train)

# Predict
rf_predictions = rf_random_search.predict(X_test)

# Evaluate
r2_rf = r2_score(y_test, rf_predictions)
mae_rf = mean_absolute_error(y_test, rf_predictions)
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_predictions))

# Print results
print("Random Forest Best Parameters:", rf_random_search.best_params_)
print(f"Random Forest R² Score: {r2_rf:.4f}")
print(f"Random Forest MAE: {mae_rf:.4f}")
print(f"Random Forest RMSE: {rmse_rf:.4f}")


joblib.dump(rf_random_search.best_estimator_, 'best_rf_pipeline.joblib')
